In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
os.chdir("E:/python/Bike_Project")
os.getcwd()
#Upload the CSV file
bike_predict=pd.read_csv("day.csv")

bike_predict.shape
bike_predict.columns
#Changing the data type of variables
cat_names=["season","yr","mnth","holiday","weekday","workingday","weathersit"]
for i in cat_names:
    print(i)
    bike_predict.loc[:,i]=bike_predict.loc[:,i].astype(str)
#Get column names
def get_cnames(data):
    all_cnames=[]
    num_cnames=[]
    cat_cnames=[]
    for i in data.columns:
        all_cnames.append(str(i))
        if(data[i].dtype=="object"):
            cat_cnames.append(str(i))
        else:
            num_cnames.append(str(i))
    cnames=[all_cnames,num_cnames,cat_cnames]
    return(cnames)
#get cnames   cnames[0]-all
#cnames[1]-numeric cnames
#cnames[2]-categorical cnames
cnames=get_cnames(bike_predict)


#get rows and columns
rows=bike_predict.shape[0]
columns=bike_predict.shape[1]

#Missing_Value_Analysis
missing_value=pd.DataFrame(bike_predict.isnull().sum())
missing_value
#No missing value is present in the data.

#Load the library
import seaborn as sn
#Visualizing the Raw Data


transformed = pd.melt(bike_predict [["yr","casual","registered"]], id_vars=['yr'], value_vars=['casual', 'registered'])
cnt_by_user= pd.DataFrame(transformed.groupby(["yr","variable"],sort=True)["value"].mean()).reset_index()
sn.pointplot(x=cnt_by_user["yr"], y=cnt_by_user["value"],hue=cnt_by_user["variable"],hue_order=["casual","registered"], data=transformed, join=True,ax=ax6)
ax6.set(xlabel='Year', ylabel='Users Count',label='big')
#Histograms before removing outliers
a4_dims = (14, 14)
fig, axes = plt.subplots(nrows=3,ncols=2,figsize=a4_dims)
sn.set(color_codes=True)
sn.set(style="white", palette="muted")
sn.distplot(bike_predict['hum'], ax=axes[0][0])
sn.distplot(bike_predict['atemp'], ax=axes[0][1])
sn.distplot(bike_predict['registered'], ax=axes[1][0])
sn.distplot(bike_predict['windspeed'], ax=axes[1][1])
sn.distplot(bike_predict['cnt'], ax=axes[2][0])
sn.distplot(bike_predict['casual'], ax=axes[2][1])

#PLotting Scatter Plot
#Visualizing distribution 
ax8 =bike_predict.plot(kind='scatter',x='instant', y='registered')
ax9 = bike_predict.plot(kind='scatter',x='instant', y='casual')
ax10 = bike_predict.plot(kind='scatter',x='instant', y='cnt')

#Outlier Analysis
out_data=bike_predict.loc[:,cnames[1]]
out_data.drop(["cnt"], axis = 1, inplace = True )
onames=out_data.columns
onames
for i in onames:
    q75,q25=np.percentile(bike_predict.loc[:,i],[75,25])
    iqr=q75-q25
    min=q25 - (1.5*iqr)
    max=q75 + (1.5*iqr)
    bike_predict.loc[bike_predict[i]<min,i]=np.nan
    bike_predict.loc[bike_predict[i]>max,i]=np.nan

# #Calculating missing value after outlier analysis
missing_val = pd.DataFrame(bike_predict.isnull().sum())
missing_val

#Impute missing  values using median
bike_predict['hum']=bike_predict['hum'].fillna(bike_predict['hum'].median())
bike_predict['windspeed']=bike_predict['windspeed'].fillna(bike_predict['windspeed'].median())
bike_predict['casual']=bike_predict['casual'].fillna(bike_predict['casual'].median())

#Histograms After Outlier Analysis
a4_dims = (14, 14)
fig, axes = plt.subplots(nrows=3,ncols=2,figsize=a4_dims)
sn.set(color_codes=True)
sn.set(style="white", palette="muted")
sn.distplot(bike_predict['hum'], ax=axes[0][0])
sn.distplot(bike_predict['atemp'], ax=axes[0][1])
sn.distplot(bike_predict['registered'], ax=axes[1][0])
sn.distplot(bike_predict['windspeed'], ax=axes[1][1])
sn.distplot(bike_predict['cnt'], ax=axes[2][0])
sn.distplot(bike_redict['casual'], ax=axes[2][1])


#Feature Selection
#Correlation Analysis
cor=cnames[1]
bike_predict_corr=bike_preddict.loc[:,cor]
f,ax=plt.subplots(figsize=(7,5))
corr=bike_predict_corr.corr()
import seaborn as sns
sns.heatmap(corr,mask=np.zeros_like(corr,dtype=np.bool),cmap=sns.diverging_palette(220,50,as_cmap=True),square=True,ax=ax)


#Feature Selection Using Anova Test
#ANOVA Analysis
import statsmodels.api as sm
from statsmodels.formula.api import ols


cw_lm=ols('cnt ~ C(yr)+C(holiday)+C(workingday)+ C(mnth)+C(weekday)+ C(weathersit)+C(season)', data=bike_predict).fit() 
print(sm.stats.anova_lm(cw_lm, typ=2))



#Dimensionality Reduction
bike_predict=bike_predict.drop(['weekday','instant','dteday','atemp'],axis=1)


#Feature Scaling
fea_name=['casual','registered']
for i in fea_name:
    print(i)
    bike_predict[i]=(bike_predict[i]-np.min(bike_predict[i]))/(np.max(bike_predict[i])-np.min(bike_predict[i]))


#Error metrix
from sklearn.metrics import mean_squared_error
from math import sqrt
def MAPE(y,yhat):
    print(np.mean(np.abs((y - yhat) / y))*100)



#Model Development



##Linear Regression Model
X=bike_predict.iloc[:,:-1].values
Y=bike_predict.iloc[:,11].values
#Splitting the data in train and testing


from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)



#Multiple Linear Regression:
from sklearn.linear_model import LinearRegression
lm_model=LinearRegression()
lm_model.fit(X_train,Y_train)
lm_predict=lm_model.predict(X_test)
MAPE(Y_test,lm_predict)


#Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor
DT_model=DecisionTreeRegressor()
DT_model.fit(X_train,Y_train)
DT_predict=DT_model.predict(X_test)
MAPE(Y_test,DT_predict)

#Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
RF_model=RandomForestRegressor()
RF_model.fit(X_train,Y_train)
RF_predict=RF_model.predict(X_test)
MAPE(Y_test,RF_predict)


SyntaxError: invalid syntax (<ipython-input-4-fe0d35173857>, line 4)